In [10]:
#https://ipython.readthedocs.io/en/stable/config/extensions/autoreload.html?highlight=autoreload
%load_ext autoreload
%autoreload 2

import json
import time
from datetime import datetime
import pandas as pd
from Kafka import KafkaWriter
from OpenWeatherMap import OpenWeatherMap
from confluent_kafka import Consumer

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
openWeatherMap = OpenWeatherMap()
kafkaForecast = KafkaWriter('weather.forecast')
kafkaCleaned = KafkaWriter('weather.cleaned')

def format_date(dt: str) -> str:
    return datetime.fromtimestamp(float(dt)).strftime('%d.%m.%Y %H:%M')


# load 'locations.json' into a json-object and return it
def load_locations() -> json:
    with open('locations.json', mode='r') as file:
        return json.load(file)

def get_transformed_forecast(cities: json) -> json:
    forecasts = []

    for key in cities:
        city = cities[key]
        forecast = openWeatherMap.get_forecast(city)

        value = {
            'city': forecast['city']['name'],
            'weather': []
        }

        preds = []
        for pred in forecast['list']:
            preds.append({
                'dt': pred['dt_txt'],
                'temp': pred['main']['temp']
            })
        value['weather'] = preds

        forecasts.append(value)

    return json.dumps({'cities':forecasts})

# for each location, query openWeatherMap for the 5-day forecast and 
# store the the returned values in Kafka
def collect_forecast_data() -> None:
    try:
        print("Starting collection ...")

        cities = load_locations()

        while True:
            forecasts = get_transformed_forecast(cities)
            dt = format_date(datetime.timestamp(datetime.now()))

            kafkaForecast.store(dt, forecasts)

            print(f"Weather data collected at {dt}!")

            time.sleep(60)

    except KeyboardInterrupt:
        print("... collection stopped!")

NoBrokersAvailable: NoBrokersAvailable

In [9]:
collect_forecast_data()

Starting collection ...


Node 3 connection failed -- refreshing metadata
Node 2 connection failed -- refreshing metadata


Weather data collected at 23.08.2022 17:12!


Node 3 connection failed -- refreshing metadata


... collection stopped!
